In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np 
import json

In [35]:
# pick full or smaller version of dataset
df = pd.read_csv('data/modelready_220423.csv')
# df = pd.read_csv('data/ten_percent.csv')

In [15]:
# reduce size only if we have big df 
# if len(df) > 63349: 
df = round(len(df) / 50)
df = df.loc[: df]
df.to_csv('data/ten_percent.csv')

In [36]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
print(f'tot columns = {len(df.columns)}, numeric type columns = {len(df.select_dtypes(include=numerics).columns)}' ) # not too many non-numeric columns
df.select_dtypes(include = ['object']).head(1)  # print non numeri columns

tot columns = 772, numeric type columns = 765


,publication_number,company_name,countries_in_family,publn_nr,primary_cpc,abstract,description_text
0,US-8623043-B1,"Entellus Medical, Inc.",['AU' 'EP' 'CA' 'US'],8623043,A61M29/02,A method of treating a constricted sinus passa...,RELATED APPLICATIONS \n This Application i...


In [37]:
# extract unique countries in the df
unique_values = set()
df['countries_in_family'].apply(lambda x: unique_values.update(x.strip("[]").replace("'", "").split())) 

# Create new columns for each unique value
for value in unique_values:
    # each country has a column (1 if the patent belong to the country 0 otherwise)
    df[value] = df['countries_in_family'].apply(lambda x: 1 if value in x else 0)


In [39]:
df['company_name_encoded'] = df.company_name.astype('category').cat.codes  # encode companies

# remove non-numeric columns
df = df.drop(['publication_number', 'company_name', 'countries_in_family', 'publn_nr',
       'primary_cpc', 'abstract', 'description_text'], axis = 1)

In [47]:
print(f'missing values = {df.isna().sum().sum()} ')# some missin values
df_no_missing = df.fillna(df.mean()).copy()
print(f'missing values after filling= {df_no_missing.isna().sum().sum()} ')

y = df_no_missing['commercialized']
df_no_missing.drop('commercialized', axis= 1, inplace=True)

# can't do standardization as some std = 0
# X = (df_no_missing - df_no_missing.mean()) / df_no_missing.std() # normalize data
scaler = MinMaxScaler()
df_X = scaler.fit_transform(df_no_missing)

missing values = 167072 
missing values after filling= 0 


In [48]:
X_train, X_test, y_train, y_test = train_test_split( df_X, y, test_size=0.20, random_state=42)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [52]:
(clf.predict(X_test) == y_test).sum() / len(y_test)

1.0

In [44]:
(y_test == 0).sum(), (y_test == 1).sum() # test are balanced

(7024, 5646)